<a href="https://colab.research.google.com/github/Lesiazhou/QM2/blob/main/Panel_Regression_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 11 12:58:09 2025

@author: dianecomon
"""
import pandas as pd
from project_tools import convert_to_number



# From Bathymetric full Save Amu Q and Syr Q to panel.csv
# Complete river inflow column with average inflow csv in panel.csv
# Join morphometric data for water volume, water level and water surface area in panel.csv
# Join precipitation and NDVI to panel.csv

panel_path = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Clean/panel.csv'
full_path = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Clean/full.csv'
average_ndvi_path = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Clean/average_year_ndvi.csv'

################################################################################################################################
# BATHYMETRIC
bat = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Bathymetric_full.csv'
bathymetric = pd.read_csv(bat)
bathymetric.set_index('Years', inplace=True)

salinity = bathymetric[['Salinity, g/I']].copy()
salinity.drop(salinity[(salinity.index<=1980) | (salinity.index > 2004)].index, inplace=True)

bathymetric.dropna(inplace=True)
bathymetric.drop(bathymetric[(bathymetric.index<=1980) | (bathymetric.index > 2004)].index, inplace=True)

panel = bathymetric[['Q Amu Darya river inflow, km3/year', 'Q Syr Darya river inflow, km3/year']].copy()

################################################################################################################################
# RIVER INFLOW

# Turn seasonal dataframes into calendar years
# Amudarya with monthly columns
amu_ng1 = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Water delivery Amudarya non-growing season.csv'
amudarya_ng1 = pd.read_csv(amu_ng1)
amudarya_ng1['Years'] = amudarya_ng1['Years'].apply(lambda x: int(x.split('-')[1]))
amudarya_ng1.set_index('Years', inplace=True)
for c in amudarya_ng1.columns:
    if not c in ['Actual', '%%']:
        amudarya_ng1[c] = [convert_to_number(s) for s in amudarya_ng1[c].tolist()]

amu_g1 = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Water delivery Amudarya growing season.csv'
amudarya_g1 = pd.read_csv(amu_g1)
amudarya_g1.set_index('Years', inplace=True)
for c in amudarya_g1.columns:
    if not c in ['Actual', '%%']:
        amudarya_g1[c] = [convert_to_number(s) for s in amudarya_g1[c].tolist()]

amudarya_calendar = pd.DataFrame(index=amudarya_g1.index, columns=['Amudarya average volume'])
for year in amudarya_calendar.index:
    a_growing_sum = amudarya_g1.loc[year, ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']].sum()

    a_non_growing_sum_1 = amudarya_ng1.loc[year + 1, ['Oct', 'Nov', 'Dec']].sum()
    a_non_growing_sum_2 = amudarya_ng1.loc[year, ['Jan', 'Feb', 'Mar']].sum()
    amudarya_calendar.loc[year, 'Amudarya average volume'] = a_growing_sum + a_non_growing_sum_1 + a_non_growing_sum_2
amudarya_calendar['Amudarya average volume'] = amudarya_calendar['Amudarya average volume'] * 0.001


# Syrdarya with monthly columns
syr_ng1 = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Water delivery Syrdarya non-growing season.csv'
syrdarya_ng1 = pd.read_csv(syr_ng1)
syrdarya_ng1['Years'] = syrdarya_ng1['Years'].apply(lambda x: int(x.split('-')[1]))
syrdarya_ng1.set_index('Years', inplace=True)
for c in syrdarya_ng1.columns:
    if not c in ['Actual', '%%']:
        syrdarya_ng1[c] = [convert_to_number(s) for s in syrdarya_ng1[c].tolist()]

syr_g1 = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Water delivery Syrdarya growing season.csv'
syrdarya_g1 = pd.read_csv(syr_g1)
syrdarya_g1.set_index('Years', inplace=True)
for c in syrdarya_g1.columns:
    if not c in ['Actual', '%%']:
        syrdarya_g1[c] = [convert_to_number(s) for s in syrdarya_g1[c].tolist()]

syrdarya_calendar = pd.DataFrame(index=syrdarya_g1.index, columns=['Syrdarya average volume'])
for year in syrdarya_calendar.index:
    s_growing_sum = syrdarya_g1.loc[year, ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']].sum()

    s_non_growing_sum_1 = syrdarya_ng1.loc[year + 1, ['Oct', 'Nov', 'Dec']].sum()
    s_non_growing_sum_2 = syrdarya_ng1.loc[year, ['Jan', 'Feb', 'Mar']].sum()
    syrdarya_calendar.loc[year, 'Syrdarya average volume'] = s_growing_sum + s_non_growing_sum_1 + s_non_growing_sum_2
syrdarya_calendar['Syrdarya average volume'] = syrdarya_calendar['Syrdarya average volume'] * 0.001

panel['Years'] = panel.index


amudarya_filtered = amudarya_calendar.loc[[2002, 2003, 2004],:]
syrdarya_filtered = syrdarya_calendar.loc[[2002, 2003, 2004], :]

amudarya_filtered = amudarya_filtered.rename(columns={'Amudarya average volume': 'Q Amu Darya river inflow, km3/year'})
syrdarya_filtered = syrdarya_filtered.rename(columns={'Syrdarya average volume': 'Q Syr Darya river inflow, km3/year'})

rivers = amudarya_filtered.join(syrdarya_filtered, how='outer')

rivers['Years'] = rivers.index
panel = pd.concat([panel, rivers], ignore_index=True)
panel.set_index('Years', inplace=True)

################################################################################################################################
# SALINITY
panel = panel.join(salinity, how='left', rsuffix='_salinity')

################################################################################################################################
# MORPHOMETRIC
morph = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/morphometric_clean.csv'
morphometric = pd.read_csv(morph)

# ensuring no index conflict
morphometric.reset_index()
morphometric.set_index('Years', inplace=True)

panel = panel.join(morphometric, how='outer')


################################################################################################################################
# PRECIPITATION
rain = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/joined_season_rivers_NDVI_precipitation.csv'
precipitation = pd.read_csv(rain)
precipitation.rename(columns={'Year': 'Years'}, inplace=True)

precipitation.join(panel, on='Years', how='left')


# Check to make sure number of NaN values has not changed after joining the datasets
# x = pd.read_feather(file_NDVI_precipitation_joined)
# y=pd.read_csv('/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Clean/panel.csv')
# y.set_index('Years', inplace=True)
# z=x.join(y,on='Year',how='left')

# print('Before join:\n\t%d non-nan\n\t%d nan values' % (x[~pd.isnull(x.NDVI)].shape[0], x[pd.isnull(x.NDVI)].shape[0]))
# print('After join:\n\t%d non-nan\n\t%d nan values' % (z[~pd.isnull(x.NDVI)].shape[0], z[pd.isnull(x.NDVI)].shape[0]))
# just_non_nan = precipitation[~pd.isnull(precipitation.NDVI)]


################################################################################################################################
# TAKE YEARLY AVERAGE FOR NDVI
avg = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/joined_season_rivers_NDVI_precipitation.csv'
average = pd.read_csv(avg)

# remove columns: 'Latitude', 'Longitude', 'Lon_Lat', 'Month', 'Growing', 'NonGrowing', 'SeasonYear', 'YearMonth', 'Date'
drop_columns = ['Unnamed: 0.1', 'Unnamed: 0', 'Latitude', 'Longitude', 'Lon_Lat', 'Month', 'Growing', 'NonGrowing', 'SeasonYear', 'YearMonth', 'Date']
average_simple = average.drop(columns=drop_columns, inplace=False)

yearly_averages = average_simple.groupby('Year')[['Precipitation', 'NDVI']].mean().reset_index()
yearly_averages.rename(columns={'Precipitation': 'Avg Precipitation', 'NDVI': 'Avg NDVI'}, inplace=True)
yearly_averages.set_index('Year', inplace=True)



################################################################################################################################
# SAVE TO COMPUTER

#panel.set_index('Years', inplace=True)

panel.to_csv(panel_path)
precipitation.to_csv(full_path)
yearly_averages.to_csv(average_ndvi_path)

print(f"Updated panel data saved to {panel_path}")
print(f"Updated precipitation data saved to {full_path}")
# print(f"Updated panel data saved to {panel_path}")


